In [ ]:
import xarray as xr
from pathlib import Path
import rioxarray
from rasterio.enums import Resampling
import pandas as pd

In [ ]:
time_series_folder = '../output_folder/completeness_analysis/'
nasa_time_series_name = '2021_SuomiNPP_nasa_time_series_fsc.nc'
meteofrance_time_series_name = '2021_meteofrance_time_series.nc'

meteofrance_time_series_path= Path(f"{time_series_folder}").joinpath(meteofrance_time_series_name)
nasa_time_series_path= Path(f"{time_series_folder}").joinpath(nasa_time_series_name)

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 0)          # No wrapping based on display width

In [ ]:
import numpy as np
from typing import Tuple, Dict
import pandas as pd
import rasterio
meteofrance_classes = {'snow_cover' : range(1,201), 'no_snow': (0,), 'clouds': (255,), 'forest_without_snow': (215,), 'forest_with_snow': (210,), 'water': (220,), 'nodata': (230,)}
nasa_classes = {'snow_cover' : range(1,101), 'no_snow': (0,), 'clouds': (250,), 'water': (237,239), 'no_decision':(201,), 'night':(211,), 'missing_data': (251,), 'L1B_unusable': (252,), 'bowtie_trim': (253,), 'L1B_fill': (254,), 'fill': (255,)}
month_dict = {'january':1, 'february':2, 'march':3, 'april':4, 'may':5, 'june':6, 'july':7, 'august':8, 'september':9, 'october':10, 'november':11, 'december':12}
nodata_nasa_classes = ('no_decision','night', 'missing_data', 'L1B_unusable', 'bowtie_trim', 'L1B_fill', 'fill')
class SnowCoverProductStatistics:
    def __init__(self, snow_cover_fraction_time_series: xr.Dataset, classes: Dict[str, Tuple[int,...] | range], nodata_mapping: Tuple[str,...] = (), mask_file: str | None = None) -> None:
        if mask_file is not None:
            self.mask = rasterio.open(mask_file).read()
        else:
            self.mask = None

        self.snow_cover_data_array = snow_cover_fraction_time_series.data_vars['snow_cover']
        if len(nodata_mapping) > 0:
            new_nodata_values = ()
            if 'nodata' in classes:
                new_nodata_values += classes['nodata']

            for class_to_exclude_name in nodata_mapping:
                for value_to_exclude in classes[class_to_exclude_name]:
                    new_nodata_values += (value_to_exclude,)
                classes.pop(class_to_exclude_name)
            classes['nodata'] = new_nodata_values
        self.classes = classes
        
        pd.options.display.float_format = '{:.3f}'.format
        pd.options.display.precision = 3


    def count_n_pixels(self, data_array: xr.DataArray)-> int:
        if self.mask is None:
            sizes = data_array.sizes
            return sizes['lon']* sizes['lat']* sizes['time']
        else:
            return data_array.count().values
    
    def compute_number_pixels_of_class(self, class_name: str, data_array: xr.DataArray):
        if type(self.classes[class_name]) == range:
            return self.compute_number_of_pixels_in_range(self.classes[class_name], data_array) 
        else:
            summed_pixels = 0
            for value in self.classes[class_name]:
                summed_pixels += self.compute_number_of_pixels_of(value, data_array)
            return summed_pixels 
    
    def compute_percentage_of_class(self, class_name: str, data_array: xr.DataArray, n_pixels_tot: int):
        return self.compute_number_pixels_of_class(class_name, data_array) / n_pixels_tot * 100
        
    
    def compute_number_of_pixels_of(self, value: int, data_array: xr.DataArray):
        return np.sum(data_array == value).values
    

    def compute_number_of_pixels_in_range(self, range: range, data_array: xr.DataArray):
        return np.sum((data_array >= range[0]) * (data_array <= range[-1])).values
    

    def _all_statistics(self, data_array: xr.DataArray, exclude_nodata: bool=False) -> Dict[str, float]:
        statistics_dict : Dict[str, float] = {}
        if exclude_nodata:
            n_pixels_tot = self.count_n_pixels(data_array) - self.compute_number_pixels_of_class(class_name='nodata',data_array= data_array)
            for class_name in self.classes:
                if class_name == 'nodata':
                    continue
                statistics_dict[class_name]=self.compute_percentage_of_class(class_name, data_array, n_pixels_tot)
        else:
            n_pixels_tot = self.count_n_pixels(data_array)
            for class_name in self.classes:
                statistics_dict[class_name]=self.compute_percentage_of_class(class_name, data_array, n_pixels_tot)
        return statistics_dict
    
    def monthly_statics(self, month: str, exclude_nodata: bool=False)-> pd.DataFrame:
        
        if self.mask is None:
            monthy_data_array = self.snow_cover_data_array.groupby('time.month')[month_dict[month]]
        else:
            monthy_data_array = self.snow_cover_data_array.groupby('time.month')[month_dict[month]].where(self.mask)
        
        statistics: Dict[str, float]  = self._all_statistics(monthy_data_array, exclude_nodata=exclude_nodata)
        statistics['n_images'] = int(monthy_data_array.sizes['time'])
        return pd.DataFrame(data=statistics.values(), index=statistics.keys())


    def year_statistics(self, exclude_nodata: bool=False):
        dfs = []
        for month in month_dict:
            print(f"Processing month {month}")
            df = self.monthly_statics(month, exclude_nodata=exclude_nodata)
            df.columns=[month]
            dfs.append(df)
        return pd.concat(dfs,axis=1)
    



In [ ]:
# Not Excluding no_data
meteofrance_statistics_calculator = SnowCoverProductStatistics(xr.open_dataset(meteofrance_time_series_path), classes=meteofrance_classes)
print(f'MeteoFrance {meteofrance_time_series_path}')
year_statistics = meteofrance_statistics_calculator.year_statistics(exclude_nodata=False)
year_statistics

In [ ]:
# Excluding no_data
meteofrance_statistics_calculator = SnowCoverProductStatistics(xr.open_dataset(meteofrance_time_series_path), classes=meteofrance_classes, mask_file='../data/vectorial/massifs_WGS84/massifs_WGS84/massifs_mask_eofr62.tiff')
print(f'MeteoFrance {meteofrance_time_series_path}')
meteofrance_year_statistics = meteofrance_statistics_calculator.year_statistics(exclude_nodata=True)
meteofrance_year_statistics.to_csv(f'../output_folder/completeness_analysis/meteo_france_massifs_{meteofrance_time_series_name}.csv')
meteofrance_year_statistics

In [ ]:
nasa_statistics_calculator = SnowCoverProductStatistics(xr.open_dataset(nasa_time_series_path), classes=nasa_classes, nodata_mapping=nodata_nasa_classes, mask_file='../data/vectorial/massifs_WGS84/massifs_WGS84/massifs_mask_v10_epsg4326.tiff')
print(f'NASA {nasa_time_series_path}')
nasa_year_statistics = nasa_statistics_calculator.year_statistics(exclude_nodata=False)
nasa_year_statistics.to_csv(f'../output_folder/completeness_analysis/nasa_massifs_{nasa_time_series_name}.csv')
nasa_year_statistics

In [ ]:
winter_months_list = ['january', 'february',	'march', 'april','may','june','november', 'december']
meteofrance_winter_stats = meteofrance_year_statistics[winter_months_list]
nasa_winter_stats = nasa_year_statistics[winter_months_list]
meteofrance_winter_stats.loc['snow_including_forest'] = meteofrance_winter_stats.loc[['snow_cover','forest_with_snow']].sum()
meteofrance_winter_stats.loc['no_snow_including_forest'] = meteofrance_winter_stats.loc[['no_snow','forest_without_snow']].sum()
meteofrance_winter_stats.loc[f'% snow_diff_wrt_nasa'] = (meteofrance_winter_stats.loc['snow_cover'] - nasa_winter_stats.loc['snow_cover']) / nasa_winter_stats.loc['snow_cover'] * 100
meteofrance_winter_stats.loc[f'% snow_diff_wrt_nasa_including_forest'] = (meteofrance_winter_stats.loc['snow_including_forest'] - nasa_winter_stats.loc['snow_cover']) / nasa_winter_stats.loc['snow_cover'] * 100
print("Meteo-France mean class distribution excluding summer\n",meteofrance_winter_stats.loc[['snow_cover', 'clouds', 'no_snow', 'snow_including_forest', 'no_snow_including_forest']].mean(axis=1))
print("NASA mean class distribution excluding summer\n",nasa_winter_stats.loc[['snow_cover', 'clouds', 'no_snow']].mean(axis=1))

In [ ]:
# DO NOT CANCELLLL
meteofrance_winter_stats.iloc[[-2,-1]]

In [ ]:
ls ../output_folder/completeness_analysis/

In [ ]:
df

In [ ]:
da.sel(stats='snow_cover').plot.step()

In [ ]:
da.sel(stats='snow_cover').plot.hist()

In [ ]:
winter_months_list = ['january', 'february',	'march', 'april','may','june','november', 'december']
meteofrance_winter_stats = meteofrance_year_statistics[winter_months_list]
nasa_winter_stats = nasa_year_statistics[winter_months_list]
total_surface = 60062.0

# Cloud cover mean
print(f'MeteoFrance Clouds % excluding summer (july - october): ',meteofrance_winter_stats.loc['clouds'].mean())
print(f'NASA Clouds % excluding summer (july - october): ',nasa_winter_stats.loc['clouds'].mean())

# Snow cover 
print(f'MeteoFrance snow covered % excluding summer (july - october): ', meteofrance_winter_stats.loc['snow_cover'].mean())
print(f'NASA snow covered % excluding summer (july - october): ', nasa_winter_stats.loc['snow_cover'].mean())

# Snow cover including forest
print(f'MeteoFrance snow covered % including forest excluding summer (july - october): ', meteofrance_winter_stats.loc['snow_cover'].mean() + meteofrance_winter_stats.loc['forest_with_snow'].mean())
print(f'NASA snow covered % including forest excluding summer (july - october): ', nasa_winter_stats.loc['snow_cover'].mean()) 


# Land including forest
print(f'MeteoFrance snow covered % including forest excluding summer (july - october): ', meteofrance_winter_stats.loc['no_snow'].mean() + meteofrance_winter_stats.loc['forest_without_snow'].mean())
print(f'NASA snow covered % including forest excluding summer (july - october): ', nasa_winter_stats.loc['no_snow'].mean()) 

In [ ]:
ls ../data/vectorial/massifs_WGS84/massifs_WGS84/

In [ ]:
import geopandas as gpd

gpd.read_file('../data/vectorial/massifs_WGS84/massifs_WGS84/massifs_WGS84.geojson')['superficie'].sum()

In [ ]:
import rasterio
import xarray as xr
import numpy as np
test_da = xr.open_dataset(meteofrance_time_series_path).data_vars['snow_cover'].groupby('time.month')[1]
r = rasterio.open('../data/vectorial/massifs_WGS84/massifs_WGS84/massifs_mask_eofr62.tiff')
masked = test_da.where(r.read(1)).where(test_da != 230)

In [ ]:
ls ../data/vectorial/massifs_WGS84/massifs_WGS84/

In [ ]:
# Old statistics 2023 showed to Simon...it's the union of NASA h18v04 and meteofrance footprints 

meteofrance_statistics_calculator = SnowCoverProductStatistics(xr.open_dataset(meteofrance_time_series_name))
print('MeteoFrance')
meteofrance_statistics_calculator.year_statistics()


In [ ]:
nasa_statistics_calculator = SnowCoverProductStatistics(nasa_ds_cropped,masked=np.nan,water=(237,220))
print('NASA')
nasa_statistics_calculator.print_year_statistics()

In [ ]:
nasa_time_series = xr.open_dataset(Path(time_series_folder, nasa_time_series_name))
nasa_time_series = nasa_time_series.rio.write_crs(nasa_time_series.data_vars['spatial_ref'].attrs['spatial_ref'])
new_nasa=nasa_time_series
meteofrance_time_series = xr.open_dataset(Path(time_series_folder, meteofrance_time_series_name))

Different results for the following two techniques

In [ ]:
# Reproject THEN resample
reprojected_nasa = nasa_time_series.rio.reproject(meteofrance_time_series.data_vars['spatial_ref'].attrs['spatial_ref'])
#reprojected_nasa.to_netcdf(Path(time_series_folder, '2017_01_nasa_time_series_reprojected.nc'), encoding={'snow_cover':{'zlib': True}, 'time': {'calendar': 'gregorian', 'units': 'days since 2016-10-01'}})
#resampled_nasa = reprojected_nasa.rio.reproject(reprojected_nasa.coords['spatial_ref'].attrs['spatial_ref'], shape=(meteofrance_time_series.sizes['lat'],meteofrance_time_series.sizes['lon']) , resampling = Resampling.bilinear)
#resampled_nasa.to_netcdf(Path(time_series_folder, '2017_01_nasa_time_series_reprojected_then_resampled.nc'), encoding={'snow_cover':{'zlib': True}, 'time': {'calendar': 'gregorian', 'units': 'days since 2016-10-01'}})
new_nasa = reprojected_nasa

In [ ]:
reprojected_nasa.to_netcdf(Path(time_series_folder, '2023_nasa_time_series_fsc_reprojected.nc'),encoding={'snow_cover':{'zlib': True}, 'time': {'calendar': 'gregorian', 'units': 'days since 2016-10-01'}})

In [ ]:
# Reproject AND resample at once
# rr_nasa = nasa_time_series.rio.reproject(meteofrance_time_series.data_vars['spatial_ref'].attrs['spatial_ref'], shape=(meteofrance_time_series.sizes['lat'],meteofrance_time_series.sizes['lon']) , resampling = Resampling.bilinear)
# rr_nasa.to_netcdf(Path(time_series_folder, '2017_01_nasa_time_series_rr.nc'), encoding={'snow_cover':{'zlib': True}, 'time': {'calendar': 'gregorian', 'units': 'days since 2016-10-01'}})

In [ ]:
lat_min = max(meteofrance_time_series.coords['lat'].min(), new_nasa.coords['y'].min())
lat_max = min(meteofrance_time_series.coords['lat'].max(), new_nasa.coords['y'].max())
lon_min = max(meteofrance_time_series.coords['lon'].min(), new_nasa.coords['x'].min())
lon_max = min(meteofrance_time_series.coords['lon'].max(), new_nasa.coords['x'].max())
lat_min, lat_max, lon_min, lon_max = [coord.values for coord in (lat_min, lat_max, lon_min, lon_max )]

In [ ]:
meteofrance_ds_cropped = meteofrance_time_series.sel(lat=slice(lat_max, lat_min), lon=slice(lon_min, lon_max))
nasa_ds_cropped = new_nasa.sel(y=slice(lat_max, lat_min), x=slice(lon_min, lon_max))
nasa_ds_cropped=nasa_ds_cropped.rename({"x": "lon", "y": "lat"})